In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import os
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
import string
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

In [3]:
import joblib
import os
os.chdir('..')

os.getcwd()

'C:\\Users\\usuario\\Desktop\\SERGIO\\Master_Data_Analytics_Inteligence\\Subjects\\MasterTrack2_Machine_learning_y_PLN\\MIAD_ML_NLP_2023'

In [4]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [5]:
def clean_text(text):
    text = re.sub((r'[^\w\s]'),'', text).lower() 
    text = re.sub((r'\d+'),'', text).lower()
    text = re.sub((r'_+'),'', text).lower()
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    words = re.sub(r'[^\w\s]', ' ', text).split()
    return ' '.join([wnl.lemmatize(word) for word in words if word not in stopwords])

dataTraining['plot'] = dataTraining['title']  +  ' ' +  dataTraining['plot']
dataTraining['clean_plot'] = dataTraining['plot'].apply(clean_text)

# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

X_train, X_test, y_train_genres, y_test_genres = train_test_split(dataTraining['clean_plot'], y_genres, test_size=0.2, random_state=3)
tfidf_vectorizer = TfidfVectorizer() 
xtrain_tfidf = tfidf_vectorizer.fit_transform(X_train)
xtest_tfidf = tfidf_vectorizer.transform(X_test)

lr = LogisticRegression(multi_class='multinomial')
clf = OneVsRestClassifier(lr)


clf.fit(xtrain_tfidf, y_train_genres)

OneVsRestClassifier(estimator=LogisticRegression(multi_class='multinomial'))

In [6]:
y_pred_genres_lr = clf.predict(xtest_tfidf)

In [7]:
joblib.dump(clf, 'Proyecto_Eniver/genero_peliculas.pkl', compress=3)

['Proyecto_Eniver/genero_peliculas.pkl']

In [8]:
joblib.dump(tfidf_vectorizer, 'Proyecto_Eniver/tfidfvectorizer.pkl')

['Proyecto_Eniver/tfidfvectorizer.pkl']

In [9]:
joblib.dump(le, 'Proyecto_Eniver/mlb.pkl')

['Proyecto_Eniver/mlb.pkl']